- 本章深入探讨事件时间（event-time）和有状态处理（stateful processing）
- 事件时间意味着：我们根据记录创建时间而非处理时间来分析信息

<h3>事件时间</h3>

- 事件时间，重要概念，而Spark的DStream API不支持有关事件时间的处理
- 以下首先介绍两种时间

<h4>事件时间</h4>

- 事件时间是嵌入在数据本身的时间，也是事件实际发生的时间
- 事件时间面临的挑战是事件数据可能会延迟或乱序

<h4>处理时间</h4>

- 处理系统实际接收数据的时间，不重要

<h4>事件时间基础知识</h4>

In [ ]:
spark.conf.set("spark.sql.shuffle.partitions", 5)
static = spark.read.json("/data/activity-data")
streaming = spark.readStream.schema(static.schema)\
    .option("maxFilesPerTrigger", 10)\
    .json("/data/acitiviy-data")

In [ ]:
streaming.printSchema()

- 此数据集中，有两个基于时间的列
- Creation_Time为事件的创建时间，Arrival_Time为事件从上游某处到达服务器的时间

<h4>事件时间的窗口</h4>

- 事件时间第一步是将时间戳转换为合适的Spark SQL时间戳类型，目前的列是以纳秒为单位（表示为long长整型）

In [ ]:
withEventTime = streaming.selectExpr(
    "*","cast(cast(Creation_Time as double)/1000000000 as timestamp) as event_time")

<h4>滚动窗口</h4>

- 最简单的时间窗口操作是计算给定时间窗口中某事件发生次数
- 如下计算基于输入数据和键执行简单求和
- 以下计算以10分钟为窗口的数据计数

In [1]:
from pyspark.sql.functions import window, col

In [ ]:
withEventTime.groupBy(window(col("event_time"), "10 minutes")).count()\
    .writeStream\
    .queryName("pyevents_per_window")\
    .format("memory")\
    .outputMode("complete")
    .start()

- 以上把数据写到了内存接收器以便于调试，可以在运行流处理之后使用SQL查询它

In [ ]:
spark.sql("SELECT * FROM events_per_window").printSchema()

- 注意时间窗口实际上是一个结构体（一个复杂类型）
- 查询该结构体以获得特定时间窗口的开始时间和结束时间

- 当然也可以对多个列进行聚合，以下就对事件时间列和用户列聚合

In [ ]:
from pyspark.sql.functions import window, col
withEventTime.groupBy(window(col("event_time"), "10 minutes"), "User").count()\
    .writeStream\
    .queryName("pyevents_per_window")\
    .format("memory")\
    .outputMode("complete")\
    .start()

<h4>滑动窗口</h4>

- 虽然是给定窗口计数，但多个窗口之间可以重叠
- 比如以下设置10分钟的窗口，每五分钟开始

In [3]:
from pyspark.sql.functions import window, col

In [ ]:
withEventTime.groupBy(window(col("event_time"), "10 minutes", "5 minutes"))\
    .count()\
    .writeStream\
    .queryName("pyevents_per_window")\
    .format("memory")\
    .outputMode("complete")\
    .start()

In [ ]:
spark.sql("SELECT * FROM events_per_window")

<h4>使用水位延迟处理数据</h4>

- 前面的例子有一个缺陷，它们没有指定系统可以接受延迟多久的吃到数据
- 因此必须指定水位，水位是给定事件或事件集之后的一个时间长度
- 在该时间长度之后，我们就不希望看到来自该时间长度之前的任何数据
- 这种数据延迟可能是由于网络延迟、设备断开连接引起的
- DStream API中没有这样一种处理延迟数据的可靠方法
- 以下举例

In [4]:
from pyspark.sql.functions import window, col

In [ ]:
withEventTime\
    .withWatermark("event_time", "30 minutes")\
    .groupBy(window(col("event_time"), "10 minutes", "5 minutes"))\
    .count()\
    .writeStream\
    .queryName("pyevents_per_window")\
    .format("memory")\
    .outputMode("complete")\
    .start()

- 这个查询语句的配置是
    - 结构化流处理会等待10分钟窗口中最晚时间戳之后30分钟
    - 每隔5分钟输出10分钟窗口
    - 完整输出模式随时间的推移而持续更新
- 如果没有指定要接受迟到多晚的数据，那么Spark会将这些数据保留在内存中
- 指定水位可使其从内存中释放

<h4>在流中删除重复项</h4>

- 重复项删除是许多应用程序需要支持的一个重要能力
- 当消息可能从上游系统多次传递时，比如物联网的上游产品在非稳定的网络环境中生产消息，相同的消息可能被多次发送
- 在这种情况下，下游应用程序和聚合操作需要保证每个消息只出现一次

In [5]:
from pyspark.sql.functions import expr

In [ ]:
withEventTiime\
    .withWatermark("event_time", "5 seconds")\
    .dropDuplicates(["User", "event_time"])\
    .groupBy("User")\
    .count()\
    .writeStream\
    .queryName("pydeduplicated")\
    .format("memory")\
    .outputMode("complete")\
    .start()

<h3>生产中的结构化流处理</h3>

<h4>容错和检查点</h4>

- 在集群发生故障时，结构化流处理允许仅通过重新启动应用程序来恢复它
- 因此，必须将应用程序配置为使用检查点和预写日志，两者由引擎自动处理
- 我们要配置检查点位置
- 在失败的情况下，只需要重新启动应用程序，确保指向相同的检查点位置
- Spark将自动恢复其状态并在其中断的位置重新开始处理数据

- 要使用检查点，需要在启动应用程序之前通过writeStream上的checkpointLocation选项指定检查点位置

In [ ]:
static = spark.read.json("/data/activity-data")
streaming = spark.readStream\
    .schema(static.schema)\
    .option("maxFilesPerTrigger", 10)\
    .json("/data/activity-data")\
    .groupBy("gt")\
    .count()

In [ ]:
query = streaming\
    .writeStream\
    .outputMode("complete")\
    .option("checkpointLocation", "/some/python/location/")\
    .queryName("test_python_stream")\
    .format("memory")\
    .start()

<h4>度量与监视</h4>

- 相比于一般Spark应用程序的度量（metrics）与监视（Monitoring）工具，结构化流处理增加了更多的选项
- 有两个关键API，分别是查询状态和近期进度

<h4>查询状态</h4>

- 我的流正在执行什么处理？
- 使用query.status命令即可返回流的当前状态

<h4>近期进度</h4>

- 虽然查询当前状态很有用，但查看查询执行进度的能力同样重要
- 进度API（progress API）允许回答
    - 处理元组的速度怎么样
    - 元组从数据源到达的速度有多快
- 运行query.recentProgress

<h4>输入速率和处理速率</h4>

- 输入速率（input rate）是指数据从输入源流向结构化流处理系统的速度
- 处理速率（process rate）是应用程序处理分析数据的速度
    - 理想情况下，输入速率和处理速率应变化一致
    - 如果输入速率远大于处理速率，说明流处理延迟落后了，需要扩招集群以处理更大的负载